In [1]:
import os

from energyflow.datasets import mod
import numpy as np

import utils

## Table 2

In [2]:
cross_sections = np.load(utils.path('sim', 'CrossSections.pickle'), allow_pickle=True)
cms_filepaths = utils.get_filenames(subdir='npz', include_path=True)
sim_filepaths = utils.get_sim_filenames_dict(subdir='sim_npz', include_path=True)
gen_filepaths = utils.get_sim_filenames_dict(subdir='gen_npz', include_path=True)
ptmins = list(sim_filepaths.keys())

In [3]:
counts = {
    'sim': {ptmin: {'nevs': 0, 'nevs_valid': 0, 'nevs_trig_fired': 0, 'njets': 0, 
                    'njets_pass_ptmin': 0, 'njets_ak5_match': 0} for ptmin in sim_filepaths},
    'cms': {'nevs': 0, 'nevs_valid': 0, 'nevs_trig_fired': 0, 'njets': 0, 
            'njets_pass_ptmin': 0, 'njets_ak5_match': 0},
    'gen': {ptmin: {'njets_gen': 0, 'njets_gen_pass_ptmin': 0} for ptmin in gen_filepaths}
}

for ptmin,filepaths in sim_filepaths.items():
    for filepath in filepaths:
        f_npz = np.load(filepath)
        counts['sim'][ptmin]['nevs'] += f_npz['nevs']
        counts['sim'][ptmin]['nevs_valid'] += f_npz['nevs_valid']
        counts['sim'][ptmin]['nevs_trig_fired'] += f_npz['nevs_trig_fired']
        counts['sim'][ptmin]['njets'] += f_npz['njets']
        counts['sim'][ptmin]['njets_pass_ptmin'] += f_npz['njets_pass_ptmin']
        counts['sim'][ptmin]['njets_ak5_match'] += f_npz['njets_ak5_match']
        f_npz.close()
    
for filepath in cms_filepaths:
    f_npz = np.load(filepath)
    counts['cms']['nevs'] += f_npz['nevs']
    counts['cms']['nevs_valid'] += f_npz['nevs_valid']
    counts['cms']['nevs_trig_fired'] += f_npz['nevs_trig_fired']
    counts['cms']['njets'] += f_npz['njets']
    counts['cms']['njets_pass_ptmin'] += f_npz['njets_pass_ptmin']
    counts['cms']['njets_ak5_match'] += f_npz['njets_ak5_match']
    f_npz.close()
    
for ptmin,filepaths in gen_filepaths.items():
    for filepath in filepaths:
        f_npz = np.load(filepath)
        counts['gen'][ptmin]['njets_gen'] += f_npz['njets_gen']
        counts['gen'][ptmin]['njets_gen_pass_ptmin'] += f_npz['njets_gen_pass_ptmin']
        f_npz.close()

In [14]:
def make_table2(out=None):
    
    # format table 2
    table_2 = ['equals', 'cols', 'equals',
                 '120',
               'hline',
                 '170', '300', '470', '600', '800', '1000', '1400', '1800',
               'equals']

    for name in table_2:
        if name == 'hline':
            print(62*'-', file=out)
        elif name == 'equals':
            print(62*'=', file=out)
        elif name == 'cols':
            cols = ('pTmin - pTmax [GeV]', 'Num. Files', 'Events', 'sigma_eff^MC [fb]')
            print('{:22}{:>10}{:>10}{:>20}'.format(*cols), file=out)
        else:
            nevs_ptmin = counts['sim'][name]['nevs']
            nfiles_ptmin = len(sim_filepaths[name])
            xs = cross_sections[name]['xs']*10**3
            
            assert nevs_ptmin == cross_sections[name]['nev']

            # format printing
            i = ptmins.index(name)
            name = '{}-{}'.format(name, ptmins[i+1] if i < len(ptmins) - 1 else 'inf')
            vals = (name, nfiles_ptmin, nevs_ptmin, xs)
            print('{:22}{:>10}{:>10}{:>20.5f}'.format(*vals), file=out)

make_table2()

with open('../plots/Table_2.txt', 'w') as f:
    make_table2(out=f)

pTmin - pTmax [GeV]   Num. Files    Events   sigma_eff^MC [fb]
120-170                      334   5963264     115133500.00000
--------------------------------------------------------------
170-300                      387   5975592      24262830.00000
300-470                      382   5975016       1168494.00000
470-600                      274   3967154         70224.21000
600-800                      271   3988701         15553.74000
800-1000                     295   3945269          1843.68800
1000-1400                    131   1956893           332.10520
1400-1800                    182   1991792            10.87214
1800-inf                      75    996500             0.35746


## Table 3

In [4]:
counts['sim']['total'] = {}
for i,ptmin in enumerate(list(counts['sim'].keys())):
    if ptmin == '120' or ptmin == 'total':
        continue
        
    for name,c in counts['sim'][ptmin].items():
        if i == 1:
            counts['sim']['total'][name] = c
        else:
            counts['sim']['total'][name] += c
            
counts['gen']['total'] = {}
for i,ptmin in enumerate(list(counts['gen'].keys())):
    if ptmin == '120' or ptmin == 'total':
        continue
        
    for name,c in counts['gen'][ptmin].items():
        name = name.replace('_gen', '')
        if i == 1:
            counts['gen']['total'][name] = c
        else:
            counts['gen']['total'][name] += c

In [5]:
def make_table3(out=None):
    
    # format table 3
    table_3 = ['equals', 'cols', 'equals',
                 'nevs',
                 'nevs_valid',
                 'present',
                 'nevs_trig_fired',
               'hline',
                 'njets',
                 'njets_pass_ptmin',
                 'njets_ak5_match',
               'equals']

    for name in table_3:
        if name == 'hline':
            print(46*'-', file=out)
        elif name == 'equals':
            print(46*'=', file=out)
        elif name == 'cols':
            cols = ('', 'CMS', 'SIM', 'GEN')
            print('{:16}{:>10}{:>10}{:>10}'.format(*cols), file=out)
        else:
            vals = (name, counts['cms'].get(name, ''), 
                    counts['sim']['total'].get(name, ''), counts['gen']['total'].get(name, ''))
            print('{:16}{:>10}{:>10}{:>10}'.format(*vals), file=out)

make_table3()

with open('../plots/Table_3.txt', 'w') as f:
    make_table3(out=f)

                       CMS       SIM       GEN
nevs              26277742  28796917          
nevs_valid        26254892  28796917          
present                                       
nevs_trig_fired    4616184  22108599          
----------------------------------------------
njets              9106775  44217198  43604940
njets_pass_ptmin   1785625  35155818  35267080
njets_ak5_match    1785625  35155790          


In [2]:
# make selections (consider amount = 0.01 for quick testing)
amount = 1.0
path = '/home/pkomiske/.energyflow/'

# use this if you've previously selected custom datasets
preselected = True

In [3]:
if not preselected:
    cms = mod.load(dataset='cms', amount=amount, store_pfcs=False)
    sim = mod.load(dataset='sim', amount=amount, store_pfcs=False, store_gens=False)
    gen = mod.load(dataset='gen', amount=amount, store_gens=False)

else:
    # make sure the path points to where you saved the custom datasets
    fullpath = os.path.join(path, 'datasets/CMS2011AJets')
    cms = mod.MODDataset('cms/CMS_Jet300_pT375-infGeV', path=fullpath, store_pfcs=False)
    sim = mod.MODDataset('sim/SIM_Jet300_pT375-infGeV_noparticles', path=fullpath)
    gen = mod.MODDataset('sim/GEN_pT375-infGeV_noparticles', path=fullpath)    

In [4]:
print('CMS Selections')
print('  All', np.count_nonzero(cms.sel()))
print('  Medium Quality', np.count_nonzero(cms.sel('quality >= 2')))
print('  |eta| < 1.9', np.count_nonzero(cms.sel('quality >= 2', 'abs_jet_eta < 1.9')))
print('  pt < 425', np.count_nonzero(cms.sel('quality >= 2', 'abs_jet_eta < 1.9', 'corr_jet_pt < 425')))
print()

print('SIM Selections')
print('  All', np.count_nonzero(sim.sel()))
print('  Medium Quality', np.count_nonzero(sim.sel('quality >= 2')))
print('  |eta| < 1.9', np.count_nonzero(sim.sel('quality >= 2', 'abs_jet_eta < 1.9')))
print('  pt < 425', np.count_nonzero(sim.sel('quality >= 2', 'abs_jet_eta < 1.9', 'corr_jet_pt < 425')))
print()

print('GEN Selections')
print('  All', np.count_nonzero(gen.sel()))
print('  |eta| < 1.9', np.count_nonzero(gen.sel('abs_jet_eta < 1.9')))
print('  pt < 425', np.count_nonzero(gen.sel('abs_jet_eta < 1.9', 'corr_jet_pt < 425')))
print()

CMS Selections
  All 1785625
  Medium Quality 1731255
  |eta| < 1.9 1690984
  pt < 425 879046

SIM Selections
  All 35155790
  Medium Quality 35145175
  |eta| < 1.9 34969900
  pt < 425 2379525

GEN Selections
  All 35267080
  |eta| < 1.9 35089120
  pt < 425 2203305



## Table 4

In [18]:
# make selections (consider amount = 0.01 for quick testing)
ptmin, ptmax = 375, 425
absetamax = '1.9'
amount = -1
specs = [(ptmin, '<=corr_jet_pts<', ptmax), 'abs_jet_eta < {}'.format(absetamax), 'quality >= 2']
path = '/home/pkomiske/.energyflow/'

# use this if you've previously selected custom datasets
preselected = True

In [19]:
if not preselected:
    cms = mod.load(*specs, dataset='cms', amount=amount)
    sim = mod.load(*specs, dataset='sim', amount=amount, store_gens=False)
    gen = mod.load(*specs, dataset='gen', amount=amount)

else:
    # make sure the path points to where you saved the custom datasets
    fullpath = os.path.join(path, 'datasets/CMS2011AJets')
    cms = mod.MODDataset('cms/CMS_Jet300_pT375-infGeV', *specs, path=fullpath)
    sim = mod.MODDataset('sim/SIM_Jet300_pT{}-{}GeV'.format(ptmin, ptmax), *specs, 
                         path=fullpath, store_gens=False)
    gen = mod.MODDataset('sim/GEN_pT{}-{}GeV'.format(ptmin, ptmax), *specs, path=fullpath)    

In [20]:
cms_pids = []
for pfcs in cms.pfcs:
    cms_pids.append(pfcs[:,cms.pid])
cms_pids = np.concatenate(cms_pids)

sim_pids = []
for pfcs in sim.pfcs:
    sim_pids.append(pfcs[:,sim.pid])
sim_pids = np.concatenate(sim_pids)

In [21]:
print('CMS:', np.unique(cms_pids, return_counts=True))
print('SIM:', np.unique(sim_pids, return_counts=True))

CMS: (array([-211.,  -13.,  -11.,   11.,   13.,   22.,  130.,  211.]), array([10414733,    17453,    31444,    31297,    16779, 14102402,
        2955136, 10731634]))
SIM: (array([-211.,  -13.,  -11.,   11.,   13.,   22.,  130.,  211.]), array([30718965,    50009,    75651,    76819,    47871, 39487711,
        7509228, 31682518]))


In [22]:
# count the number of different particle types at different stages
names = ['CMS','SIM']
for k,events in enumerate([cms.pfcs, sim.pfcs]):
    print(names[k])
    
    # iterate over all of the particle types
    for pid in [11,-11,13,-13,211,-211,22,130]:
        count_tot, count_chs, count_ptc = 0, 0, 0
        for jet in events:

            # mask to restrict to this pid
            pidfilt = jet[:,4] == pid

            # mask to restrict to PFCs above 1 GeV
            ptcfilt = jet[:,0] > 1.

            # mask to restrict to post-CHS particles
            chsfilt = jet[:,5] <= 0

            # count the number of particles at each filtering stage
            count_tot += np.count_nonzero(pidfilt)
            pidchs_filt = pidfilt & chsfilt
            count_chs += np.count_nonzero(pidchs_filt)
            count_ptc += np.count_nonzero(pidchs_filt & ptcfilt)

        # print the result
        print(pid,'    --  Total:',    count_tot)
        print(pid,'    -- w. CHS:',    count_chs)
        print(pid,'-- w. pT>1GeV:',    count_ptc)
        
        # also formatted for copypasting into tex file
        print(count_tot,'&',count_chs,'&',count_ptc)

        print()
    print()

CMS
11     --  Total: 31297
11     -- w. CHS: 30304
11 -- w. pT>1GeV: 30284
31297 & 30304 & 30284

-11     --  Total: 31444
-11     -- w. CHS: 30470
-11 -- w. pT>1GeV: 30448
31444 & 30470 & 30448

13     --  Total: 16779
13     -- w. CHS: 14957
13 -- w. pT>1GeV: 14912
16779 & 14957 & 14912

-13     --  Total: 17453
-13     -- w. CHS: 15373
-13 -- w. pT>1GeV: 15310
17453 & 15373 & 15310

211     --  Total: 10731634
211     -- w. CHS: 8159520
211 -- w. pT>1GeV: 6950019
10731634 & 8159520 & 6950019

-211     --  Total: 10414733
-211     -- w. CHS: 7987681
-211 -- w. pT>1GeV: 6780597
10414733 & 7987681 & 6780597

22     --  Total: 14102402
22     -- w. CHS: 14102402
22 -- w. pT>1GeV: 7157772
14102402 & 14102402 & 7157772

130     --  Total: 2955136
130     -- w. CHS: 2955136
130 -- w. pT>1GeV: 2317806
2955136 & 2955136 & 2317806


SIM
11     --  Total: 76819
11     -- w. CHS: 73937
11 -- w. pT>1GeV: 73906
76819 & 73937 & 73906

-11     --  Total: 75651
-11     -- w. CHS: 72920
-11 -- w. pT